# MODULE 1 
## Game files: clean and import data

In this first step of our data analysis, I will read the MLB All Star game files into a pandas dataframe and clean some of the data for later analysis. 

In [1]:
import os
import glob
import pandas as pd

All of the game files live in the `games` folder with extension `.eve`. I create a list of the file names of the game files to easily import them into a dataframe later.  

In [2]:
os.chdir('/Users/stephanie/github/python-baseball/')
game_files = glob.glob(os.path.join(os.getcwd(), 'games', '*.EVE'))
game_files.sort()

Now that I have a list of the file names of the game files, I can read each of the game files into pandas dataframes, adding the game datagrames to a list, `game_frames`. Then, I concatenate the game dataframes into one dataframe, called `games`.

In [3]:
game_frames = []
for game_file in game_files: 
    game_frame = pd.read_csv(game_file, names=['type', 'multi2', 'multi3', 'multi4', 'multi5', 'multi6', 'event'])
    game_frames.append(game_frame)
    
games = pd.concat(game_frames)
games.head()

,type,multi2,multi3,multi4,multi5,multi6,event
0,id,ALS193307060,NaN,NaN,NaN,NaN,NaN
1,version,1,NaN,NaN,NaN,NaN,NaN
2,info,inputprogvers,version 7RS(19) of 07/07/92,NaN,NaN,NaN,NaN
3,info,visteam,NLS,NaN,NaN,NaN,NaN
4,info,hometeam,ALS,NaN,NaN,NaN,NaN


A quick glance at the data shows that there are many rows missing values or have unknown values. For example, in the `multi5` column, there are some rows that are filled with '??'

In [4]:
games.loc[games['multi5'] == '??']

,type,multi2,multi3,multi4,multi5,multi6,event
53,play,1,0,martp103,??,NaN,63
54,play,1,0,frisf101,??,NaN,63/G6D
55,play,1,0,kleic101,??,NaN,6/L
56,play,1,1,chapb102,??,NaN,53
57,play,1,1,gehrc101,??,NaN,W
...,...,...,...,...,...,...,...
186,play,8,1,mcgwm001,??,NaN,K
187,play,9,0,johnc002,??,NaN,NP
189,play,9,0,johnc002,??,NaN,K
190,play,9,0,gracm001,??,NaN,3/G


To keep the data consistent, I replace the rows that have values of '??' in the `multi5` column with empty strings. 

In [5]:
games.loc[games['multi5'] == '??', ['multi5']] = ''

Each row in the `games` dataframe should logically have its associated game ID. I can extract the game ID from the `multi2` column, but not rows in the `multi2` column also contains other information besides the game ID. I'll isolate the game IDs only using regex, and also extract the year that the game was played in (from the game ID). The new dataframe of game IDs and years will be `identifiers`.

In [6]:
identifiers = games['multi2'].str.extract(r'(.LS(\d{4})\d{5})')
identifiers.head()

,0,1
0,ALS193307060,1933
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN


As observed previously, not every row in the `games` df has a game ID in the `multi2` column, hence the regex returns `NaN` values in most of the rows in the `identifiers` dataframe. I want every row to have a game ID, so I forward fill the `NaN` values with the game ID that preceded that row. In other words, every row that follows a row with a game ID belongs to the same game, so it gets assigned the same game ID (until we hit the next row that has a new game ID). 

In [7]:
identifiers = identifiers.fillna(method = 'ffill') 
identifiers.head()

,0,1
0,ALS193307060,1933
1,ALS193307060,1933
2,ALS193307060,1933
3,ALS193307060,1933
4,ALS193307060,1933


Let's rename the column labels of the `identifiers` dataframe to be more descriptive. 

In [8]:
identifiers.columns = ['game_id', 'year']
identifiers.head()

,game_id,year
0,ALS193307060,1933
1,ALS193307060,1933
2,ALS193307060,1933
3,ALS193307060,1933
4,ALS193307060,1933


Lastly, I'll add the `game_id` and `year` columns to the original `games` dataframe by concatenating the `games` dataframe with the `identifiers` dataframe. 

In [9]:
games = pd.concat([games, identifiers], axis=1, sort=False) 
games.head()

,type,multi2,multi3,multi4,multi5,multi6,event,game_id,year
0,id,ALS193307060,NaN,NaN,NaN,NaN,NaN,ALS193307060,1933
1,version,1,NaN,NaN,NaN,NaN,NaN,ALS193307060,1933
2,info,inputprogvers,version 7RS(19) of 07/07/92,NaN,NaN,NaN,NaN,ALS193307060,1933
3,info,visteam,NLS,NaN,NaN,NaN,NaN,ALS193307060,1933
4,info,hometeam,ALS,NaN,NaN,NaN,NaN,ALS193307060,1933


I clean up the data in the new `games` dataframe further by replacing all of the `NaN` values with empty strings. 

In [10]:
games = games.fillna('')
games.head()

,type,multi2,multi3,multi4,multi5,multi6,event,game_id,year
0,id,ALS193307060,,,,,,ALS193307060,1933
1,version,1,,,,,,ALS193307060,1933
2,info,inputprogvers,version 7RS(19) of 07/07/92,,,,,ALS193307060,1933
3,info,visteam,NLS,,,,,ALS193307060,1933
4,info,hometeam,ALS,,,,,ALS193307060,1933


The `type` column in the `games` dataframe only contains six possible values: `info`, `start`, `play`, `com`, `sub`, `data`. I can use this to indicate data type. 

In [11]:
games.loc[:,'type'] = pd.Categorical(games.loc[:, 'type'])

I now have the final, cleaned `games` dataframe containing all of the data for MLB All Star Games. 

In [12]:
games.head()

,type,multi2,multi3,multi4,multi5,multi6,event,game_id,year
0,id,ALS193307060,,,,,,ALS193307060,1933
1,version,1,,,,,,ALS193307060,1933
2,info,inputprogvers,version 7RS(19) of 07/07/92,,,,,ALS193307060,1933
3,info,visteam,NLS,,,,,ALS193307060,1933
4,info,hometeam,ALS,,,,,ALS193307060,1933
